In [1]:
import subprocess
import pandas as pd
import chardet
import numpy as np
import os
import glob
import argparse
import hashlib
from datetime import datetime
from configparser import ConfigParser
import sys

In [2]:
# Define path and filename variables

file_path = "C:\\Users\\Rmittal\\CCS\\Internal - Analytics - Shared Drive\\1. Shared Drive\\Clients"
subdir = "Raw Client Data" 
clients = "Thomas College"
file_name = "Section 3 constituent level data 01312024.xlsx"
sheet_name = ""

gui=False
if gui:
    file_path = input("Enter file path:")
    file_name = input("Enter file name:")
    sheet_name = input("Enter sheet name if Excel spreadsheet. Hit Enter if only single sheet:")

if sheet_name=='':
    sheet_name = None

file_details = {"file_name": file_name, "sheet_name": sheet_name}
file = "%s\\%s\\%s\\%s" %(file_path, clients, subdir, file_name)
prefix = ".".join(file_name.split(".")[0:-1])

In [ ]:
detect_encoding = False
if detect_encoding:
    with open(file, 'rb') as f:
        raw_data = f.readline()
    raw_data_encoding = chardet.detect(raw_data)['encoding']
    print(raw_data_encoding)
    
    fext = file.split(".")[-1]
    if fext.lower() == "xlsx":
        df = pd.read_excel(file, file_details["sheet_name"])
    elif fext.lower() == "csv":
        df = pd.read_csv(file, encoding=raw_data_encoding)

# Define the column names and their corresponding data types
#dtype_dict = {"Donor ID": str}  # Replace 'column_with_leading_zeros' with the actual column name

df = pd.read_excel(file)
df = pd.read_excel(file, dtype={col: str for col in df.columns if 'id' in col.lower()})

In [33]:
### For the gifts file
#[(key, df[key].value_counts()) for key in df.keys() if "id" in key.lower()]
#df = df.drop(columns=columns_to_drop)
#df = df.drop(columns=["filename"])
#df.to_csv("../nsf/national_scleroderma_foundation_gifts_clean_1.csv", index=False)

In [24]:
#m_giving_logp1 = np.log1p(df["total_with_hard_soft_credit"])/np.log(10)
#m_giving_logp1.hist()

In [67]:
df = df_excel

### Preprocessing before mapping

In [38]:
run = False
if run:
    df = df.rename(columns={"zip": "zip_raw"})
    df["solicit_codes"] =  df.filter(like='solicit').notna().any(axis=1).astype(int)
    df["five_year_giving"] =  df["total_giving_2023"] + df["total_giving_2022"] + df["total_giving_2021"] \
                            + df["total_giving_2020"] + df["total_giving_2019"] 

In [224]:
#[(key, df[key].value_counts()) for key in df.keys() if "constituent" in key.lower()]

In [5]:
mapping = pd.read_csv("./thomas_college_constit_mapping.csv")
column_mapping = {row["file_columns"]: row["expected_columns"] \
                  for index, row in mapping.iterrows() if row["file_columns"]!="not_found"}
df.rename(columns=column_mapping, inplace=True)
df = df[list(column_mapping.values())]

### Preprocessing after mapping

In [92]:
df["class_year"] = df["class_year"].str.split("'",expand=True)[1]
df["class_year"] = df["class_year"].fillna(np.nan).astype('Int64')

In [93]:
#df["class_year"] = np.where(((df["class_year"] > 50) & (~df["class_year"].isna())), "19" + df["class_year"].astype(str), \
#                            "20" + df["class_year"].astype(str))
mask = ~df["class_year"].isna()
df.loc[mask, "class_year"] = np.where((df.loc[mask, "class_year"] > 25),
                                      "19" + df.loc[mask, "class_year"].astype(str),
                                      "20" + df.loc[mask, "class_year"].astype(str))


In [94]:
cols_datetime = list(df.select_dtypes(include=['datetime']).keys())
for col in cols_datetime:
    df[col] = pd.to_datetime(df[col]).dt.date

In [230]:
### Exploratory Analysis
#df.tail()
#df = df.loc[0:2674626]
df_split = np.array_split(df, 4)
[len(df_split[i]) for i in range(4)]

[62111, 62111, 62111, 62110]

### Splitting

In [231]:
def file_split(df, file, chunk_size=None):
    '''
    Splits files into parts, or in chunk_size
    '''
    if not file:
        print("Provide a File")
        return False
    if not chunk_size:
        print("Provide non-zero chunk_size in MB")
        return False
    
    ### File Attributes
    fsize = os.path.getsize(file)
    fdir, fname = os.path.split(file)
    fname = os.path.splitext(fname)[0]
    fext = file.split(".")[-1]

    ### Determine the number of parts
    chunk_size = chunk_size*1E+6
    parts = int(np.floor(fsize/chunk_size))
    
    if chunk_size > fsize:
        raise ValueError('Chunk size cannot be greater than file size')
    else:
        print("The file is being split into %d files" %parts)
        
    df_split = np.array_split(df, parts)

    for i in range(parts):
        print("part %d" %i)
        if fext.lower() == "xlsx":
            df_split[i].to_excel("%s/%s_%d.%s" %(fdir, fname, i+1, fext), index=False)
        elif fext.lower() == "csv":
            df_split[i].to_csv("%s/%s_%d.%s" %(fdir, fname, i+1, fext), index=False)       

In [102]:
#file_split(df, file, chunk_size=50)
df.to_csv("./thomas_college_constit_clean.csv", index=False)

In [250]:
#df['history_of_volunteer'] = df['history_of_volunteer'].astype(str)
#df['history_of_volunteer'].str.len().value_counts()
#df.dtypes

In [73]:
df.head()

,constituent_id,creation_date,modification_date,key_indicator,deceased,prefix,first_name,middle_name,last_name,suffix,...,first_gift_amount,first_gift_date,First Gift Fund,most_recent_gift_amount,most_recent_gift_date,Last Gift Fund,Largest Gift Amount,Largest Gift Date,Largest Gift Fund,number_of_special_events_attended
0,15197,2004-03-04,2023-01-06,Indiv,No,Ms.,Tricia,E,Abbott,NaN,...,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,NaN,0
1,11550,2004-03-04,2024-01-08,Indiv,No,Ms.,Diane,C,Abbott-Cookson,NaN,...,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,NaN,0
2,14266,2004-03-04,2024-01-08,Indiv,No,Mrs.,Arlene,Mildred,Abrams,NaN,...,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,NaN,0
3,28105,2004-03-04,2023-04-18,Indiv,No,Mr.,Robert,NaN,Abrams,NaN,...,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,NaN,0
4,17845,2004-03-04,2012-02-10,Indiv,No,Mr.,Anthony,NaN,Abreu,NaN,...,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,NaN,0


In [6]:
df_subset = df[df.groupby("constituent_id")["constituent_id"].transform('size') > 1]
df_subset